# ⚙️ Vibration Fault Classification Training (CWRU)

This notebook trains a **CNN** on the CWRU Bearing dataset.

**Supports:**
- 2D image data (spectrograms, 32x32)
- 1D raw signal data (2048 samples)
- Pre-processed .npz files
- Raw .mat files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install scipy seaborn -q

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import glob
import os
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## 1. Configuration

In [ ]:
# ============= UPDATE THIS PATH =============
DATA_PATH = '/content/drive/MyDrive/MaintanenceAI/Data/CWRU'
SAVE_PATH = '/content/drive/MyDrive/MaintanenceAI/trained_models'
# =============================================

# Hyperparameters
EPOCHS = 30
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
TEST_SPLIT = 0.2

print(f'Data path: {DATA_PATH}')

## 2. Data Loading

In [ ]:
def load_data(data_path):
    """Load data from .npz file with auto-detection of format."""
    npz_files = glob.glob(os.path.join(data_path, '*.npz'))
    if not npz_files:
        raise FileNotFoundError(f'No .npz files found in {data_path}')
    
    npz_path = npz_files[0]
    print(f'Loading: {npz_path}')
    
    data = np.load(npz_path, allow_pickle=True)
    print(f'Available keys: {list(data.keys())}')
    
    # Try common key names
    x_keys = ['X', 'x', 'data', 'signals', 'features', 'images']
    y_keys = ['y', 'Y', 'labels', 'targets']
    
    X, y = None, None
    keys = list(data.keys())
    
    for key in x_keys:
        if key in data:
            X = data[key]
            break
    if X is None:
        X = data[keys[0]]
    
    for key in y_keys:
        if key in data:
            y = data[key]
            break
    if y is None and len(keys) > 1:
        y = data[keys[1]]
    
    print(f'X shape: {X.shape}')
    print(f'y shape: {y.shape}')
    
    return X, y

# Load data
print('Loading CWRU data...\n')
X, y = load_data(DATA_PATH)

# Determine data type
if len(X.shape) == 2:
    DATA_TYPE = '1D'
    print(f'\nDetected: 1D signal data (shape: {X.shape})')
elif len(X.shape) == 3:
    DATA_TYPE = '2D'
    print(f'\nDetected: 2D image data (shape: {X.shape})')
elif len(X.shape) == 4:
    DATA_TYPE = '2D'
    print(f'\nDetected: 2D image data with channels (shape: {X.shape})')
else:
    raise ValueError(f'Unexpected data shape: {X.shape}')

# Handle labels
le = LabelEncoder()
if y.dtype == object or (len(y.shape) == 1 and isinstance(y[0], str)):
    y = le.fit_transform(y)
else:
    y = y.astype(int)
    if len(y.shape) > 1:
        y = y.flatten()
    n_classes = len(np.unique(y))
    label_names = [f'Class_{i}' for i in range(n_classes)]
    le.classes_ = np.array(label_names)

NUM_CLASSES = len(np.unique(y))
print(f'Number of classes: {NUM_CLASSES}')

In [ ]:
# Class distribution
unique, counts = np.unique(y, return_counts=True)
print('\nClass Distribution:')
for cls, cnt in zip(unique, counts):
    name = le.classes_[cls] if cls < len(le.classes_) else f'Class_{cls}'
    print(f'  {name}: {cnt} samples')

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SPLIT, random_state=42, stratify=y
)
print(f'Train: {len(X_train)}, Test: {len(X_test)}')

## 3. Model Architecture

In [ ]:
class VibrationClassifier1D(nn.Module):
    """1D CNN for raw signal data."""
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=64, stride=2, padding=32),
            nn.BatchNorm1d(16), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(16, 32, kernel_size=32, stride=2, padding=16),
            nn.BatchNorm1d(32), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=16, stride=2, padding=8),
            nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=8, stride=2, padding=4),
            nn.BatchNorm1d(128), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


class VibrationClassifier2D(nn.Module):
    """2D CNN for spectrogram/image data."""
    def __init__(self, num_classes=10, input_channels=1):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, 128), nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


# Select model based on data type
if DATA_TYPE == '1D':
    model = VibrationClassifier1D(num_classes=NUM_CLASSES).to(device)
    print('Using 1D CNN model')
else:
    # Determine input channels
    if len(X.shape) == 3:
        input_channels = 1
    else:
        input_channels = X.shape[1] if X.shape[1] in [1, 3] else 1
    model = VibrationClassifier2D(num_classes=NUM_CLASSES, input_channels=input_channels).to(device)
    print(f'Using 2D CNN model (input channels: {input_channels})')

print(f'Model parameters: {sum(p.numel() for p in model.parameters()):,}')

## 4. Dataset & DataLoader

In [ ]:
class CWRUDataset(Dataset):
    def __init__(self, data, labels, data_type='2D'):
        self.data = data
        self.labels = labels
        self.data_type = data_type

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx].astype(np.float32)
        
        # Normalize
        x = (x - np.mean(x)) / (np.std(x) + 1e-8)
        
        if self.data_type == '1D':
            # Shape: (1, signal_length)
            x = torch.tensor(x).unsqueeze(0)
        else:
            # Shape: (1, H, W) for 2D
            if len(x.shape) == 2:
                x = torch.tensor(x).unsqueeze(0)
            elif len(x.shape) == 3:
                # Already has channels, just convert
                x = torch.tensor(x)
                # If channels are last, move to first
                if x.shape[0] not in [1, 3] and x.shape[-1] in [1, 3]:
                    x = x.permute(2, 0, 1)
            else:
                x = torch.tensor(x)
        
        return x, torch.tensor(self.labels[idx], dtype=torch.long)


train_dataset = CWRUDataset(X_train, y_train, DATA_TYPE)
test_dataset = CWRUDataset(X_test, y_test, DATA_TYPE)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Verify shapes
sample_x, sample_y = next(iter(train_loader))
print(f'Batch shape: {sample_x.shape}')
print(f'Labels shape: {sample_y.shape}')

## 5. Training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

train_losses, train_accs = [], []

print('Starting Training...\n')
for epoch in range(EPOCHS):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for data, target in tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}', leave=False):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)
    
    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    train_losses.append(avg_loss)
    train_accs.append(accuracy)
    scheduler.step(avg_loss)
    
    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

print('\n✅ Training Complete!')

## 6. Evaluation

In [ ]:
model.eval()
all_preds, all_targets = [], []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_targets.extend(target.cpu().numpy())

all_preds = np.array(all_preds)
all_targets = np.array(all_targets)

test_accuracy = 100. * np.mean(all_preds == all_targets)
print(f'\n📊 Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
# Classification Report
print('\nClassification Report:')
print(classification_report(all_targets, all_preds, target_names=le.classes_))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(all_targets, all_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title(f'Confusion Matrix (Accuracy: {test_accuracy:.1f}%)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
plt.show()

## 7. Save Model

In [ ]:
os.makedirs(SAVE_PATH, exist_ok=True)
save_file = os.path.join(SAVE_PATH, 'vibration_classifier.pth')

torch.save({
    'model_state_dict': model.state_dict(),
    'label_encoder_classes': le.classes_.tolist(),
    'test_accuracy': test_accuracy,
    'num_classes': NUM_CLASSES,
    'data_type': DATA_TYPE,
    'model_type': '2D' if DATA_TYPE == '2D' else '1D'
}, save_file)

print(f'✅ Model saved: {save_file}')

In [ ]:
# Training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(train_losses)
axes[0].set_title('Training Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].grid(True)

axes[1].plot(train_accs)
axes[1].set_title('Training Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150)
plt.show()